In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [5]:
def huber_fn(y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-.5
    return tf.where(is_small_error,squared_loss,linear_loss)


In [30]:
X_train[1,:].shape

(8,)

In [36]:
input_shape = X_train[1,:].shape
model = keras.models.Sequential([
    keras.layers.Dense(30,activation = "selu", kernel_initializer = "lecun_normal",
                      input_shape=input_shape),
    keras.layers.Dense(1)
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 30)                270       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(loss=huber_fn,optimizer="nadam",metrics=["mae"])
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.6619 - mae: 1.0188 - val_loss: 0.2705 - val_mae: 0.5646
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2168 - mae: 0.5136 - val_loss: 0.2158 - val_mae: 0.5036
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2069 - mae: 0.5002 - val_loss: 0.2018 - val_mae: 0.4878
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.2020 - mae: 0.4931 - val_loss: 0.1855 - val_mae: 0.4685
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1978 - mae: 0.4869 - val_loss: 0.1853 - val_mae: 0.4701
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1951 - mae: 0.4821 - val_loss: 0.1929 - val_mae: 0.4720
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1924 - mae: 0.4780 - val_loss: 0.1907 - val_mae: 0.4659
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.

In [38]:
model.save("../Models/5_CustomModelAndTraining")

INFO:tensorflow:Assets written to: ../Models/5_CustomModelAndTraining/assets


In [39]:
model = keras.models.load_model("../Models/5_CustomModelAndTraining",
                                custom_objects={"huber_fn":huber_fn})

In [40]:
model.fit(X_train_scaled,y_train,epochs=10,validation_data=(X_valid_scaled,y_valid))

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1829 - mean_absolute_error: 0.4629 - val_loss: 0.1714 - val_mean_absolute_error: 0.4399
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1809 - mean_absolute_error: 0.4588 - val_loss: 0.1840 - val_mean_absolute_error: 0.4557
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1792 - mean_absolute_error: 0.4558 - val_loss: 0.1728 - val_mean_absolute_error: 0.4444
Epoch 4/10
363/363 [==============================] - 0s 994us/step - loss: 0.1776 - mean_absolute_error: 0.4537 - val_loss: 0.1731 - val_mean_absolute_error: 0.4411
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1751 - mean_absolute_error: 0.4488 - val_loss: 0.1830 - val_mean_absolute_error: 0.4511
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.1736 - mean_absolute_error: 0.4469 - val_loss: 0.1700 - val_mean_absolute_error: 0.4387
Epoch 7/10
363

In [46]:
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold = 1.0,**kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error)<self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold* tf.abs(error)-self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return{**base_config,"threshold":self.threshold}

In [47]:
model.compile(loss=HuberLoss(2.0),optimizer="nadam",metrics=["mae"])
model.save("../Models/5_CustomModelAndTraining_HuberClass")

INFO:tensorflow:Assets written to: ../Models/5_CustomModelAndTraining_HuberClass/assets


In [ ]:
model = ke